In [1]:
import pvlib

/Users/srijananand/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('dataSample1.csv')

In [3]:
smb_columns = ['SMB1 Total Current(A)', 'SMB2 Total Current(A)', 'SMB3 Total Current(A)', 
               'SMB4 Total Current(A)', 'SMB5 Total Current(A)', 'SMB6 Total Current(A)', 
               'SMB7 Total Current(A)']

vdc_columns = ['DC Voltage (V)']

# Create the 2D array with only SMB currents
smb_current_array = df[smb_columns].values

# Create the array with only DC voltages
vdc_array = df[vdc_columns].values

# Create the array with DC Power
pdc_array = []

for i in range(len(vdc_array)):
    p_dc_i = 0
    for j in range(len(smb_current_array[i])):
        p_dc_i += smb_current_array[i][j] * vdc_array[i]
    pdc_array.append(p_dc_i)

In [22]:
# Import necessary libraries
import numpy as np

# Given module parameters
module_parameters = {
    'Name': 'HT72-156P',
    'pdc0': 340,  # DC power output at standard test conditions (W)
    'gamma_pdc': -0.0041,  # Temperature coefficient of power (%/°C)
    'b': -0.0041,  # Temperature coefficient of power (%/°C) - alias
    'BIPV': 'N',
    'Date': '2018',
    'T_NOCT': 45,
    'A_c': 1.656 * 0.992,  # Module area in m^2
    'N_s': 72,
    'I_sc_ref': 9.62,
    'V_oc_ref': 46.6,
    'I_mp_ref': 8.92,
    'V_mp_ref': 38.2,
    'alpha_sc': 0.00050,
    'beta_oc': -0.0032,
    'gamma_r': -0.0041,
    'cells_in_series': 72
}

# Given inverter parameters
inverter_parameters = {
    'Name': 'Vacon 8000 Solar 1250 kW',
    'Vac': 400,
    'Paco': 1250000,
    'Pdco': 1268681.23,  # Corrected for efficiency
    'Vdco': 410,  # Corrected to be within the MPPT range
    'Pso': 30,  # Assuming this value based on general specs
    'C0': -0.00001,  # Assuming these values
    'C1': 0.001,
    'C2': -0.000005,
    'C3': 0,
    'Pnt': 0.1,  # Assuming this value based on general specs
    'Vdcmax': 900,
    'Idcmax': 353,
    'Mppt_low': 410,  # Corrected to match document
    'Mppt_high': 800,
    'pdc0': 1250000
}

def _sandia_eff(v_dc, p_dc, inverter):
    """
    Calculate the inverter AC power without clipping
    """
    Paco = inverter['Paco']
    Pdco = inverter['Pdco']
    Vdco = inverter['Vdco']
    C0 = inverter['C0']
    C1 = inverter['C1']
    C2 = inverter['C2']
    C3 = inverter['C3']
    Pso = inverter['Pso']

    A = Pdco * (1 + C1 * (v_dc - Vdco))
    B = Pso * (1 + C2 * (v_dc - Vdco))
    C = C0 * (1 + C3 * (v_dc - Vdco))

    return (Paco / (A - B) - C * (A - B)) * (p_dc - B) + C * (p_dc - B) ** 2

def _sandia_limits(power_ac, p_dc, Paco, Pnt, Pso):
    """
    Applies minimum and maximum power limits to `power_ac`
    """
    power_ac = np.minimum(Paco, power_ac)
    min_ac_power = -1.0 * abs(Pnt)
    if p_dc < Pso:
        power_ac = min_ac_power
    return power_ac

def sandia(v_dc, p_dc, inverter):
    """
    Convert DC power and voltage to AC power using Sandia's Grid-Connected PV Inverter model.
    """
    Paco = inverter['Paco']
    Pnt = inverter['Pnt']
    Pso = inverter['Pso']

    power_ac = _sandia_eff(v_dc, p_dc, inverter)
    print("This : ", power_ac)
    power_ac = _sandia_limits(power_ac, p_dc, Paco, Pnt, Pso)

    return power_ac

# Calculate the DC power output of the module
for i in range(600,605):
    pdc_i = 0
    pac_i = 0
    for j in range(len(smb_current_array[i])):
        pdc_i += smb_current_array[i][j] * vdc_array[i][0]
        pdc_i_j = smb_current_array[i][j] * vdc_array[i][0]
        print(f"DC Current in SMB {j+1}: {smb_current_array[i][j]:.2f} A")
        print(f"DC Power in SMB {j+1}: {pdc_i_j:.2f} W")
        print(f"DC Voltage in SMB {j+1}: {vdc_array[i][0]}")


        # Calculate AC power output per SMB
        v_dc = vdc_array[i][0]  # DC voltage input to the inverter
        p_dc = smb_current_array[i][j] * vdc_array[i][0]  # DC power input to the inverter

        power_ac_per_inverter = sandia(v_dc, p_dc, inverter_parameters)

        print(f"Per SMB AC Power Output: {power_ac_per_inverter:.2f} W")
        pac_i += power_ac_per_inverter

    # Calculate total AC power output
    print(f"Total AC Power Output : {pac_i:.2f} W")     
    print(f"Total DC Power Output : {pdc_i:.2f} W")

DC Current in SMB 1: 185.90 A
DC Power in SMB 1: 125296.60 W
DC Voltage in SMB 1: 674
This :  1949483.2599819484
Per SMB AC Power Output: 1250000.00 W
DC Current in SMB 2: 184.00 A
DC Power in SMB 2: 124016.00 W
DC Voltage in SMB 2: 674
This :  1931141.471169732
Per SMB AC Power Output: 1250000.00 W
DC Current in SMB 3: 184.20 A
DC Power in SMB 3: 124150.80 W
DC Voltage in SMB 3: 674
This :  1933073.7303199442
Per SMB AC Power Output: 1250000.00 W
DC Current in SMB 4: 184.10 A
DC Power in SMB 4: 124083.40 W
DC Voltage in SMB 4: 674
This :  1932107.646172438
Per SMB AC Power Output: 1250000.00 W
DC Current in SMB 5: 182.70 A
DC Power in SMB 5: 123139.80 W
DC Voltage in SMB 5: 674
This :  1918572.9283113524
Per SMB AC Power Output: 1250000.00 W
DC Current in SMB 6: 185.90 A
DC Power in SMB 6: 125296.60 W
DC Voltage in SMB 6: 674
This :  1949483.2599819484
Per SMB AC Power Output: 1250000.00 W
DC Current in SMB 7: 185.40 A
DC Power in SMB 7: 124959.60 W
DC Voltage in SMB 7: 674
This :  19